In [ ]:
##JSS
import sys,time
import numpy as np
import cv2
import roslib
import rospy
import h5py
from sensor_msgs.msg import Image
import common_function as cf
from pathlib import Path
import pycolmap
from collections import defaultdict
import time
import logging
import torch


## superglue ka hai yeh
import torch
import tqdm
from hloc import matchers
from hloc.utils.base_model import dynamic_load
from hloc.utils.parsers import names_to_pair 
from hloc import extractors
from hloc.utils.tools import map_tensor


## localize sfm ka hai
from hloc.utils.read_write_model_apna import read_model

## DBOW ka hai
import pyDBoW3 as bow

##superpoint ka hai
from Superpoint_class import Superpoint as sp

In [ ]:
class image_pose:
    def __init__(self,config):
        self.config = config 
        self.device = 'cuda:0'
        Model = dynamic_load(matchers, self.config["superglue_config"]["model"]["name"])
        self.superglue_model = Model(self.config["superglue_config"]["model"]).eval().to(self.device)
        voc = bow.Vocabulary()
        voc.load(self.config["superpoint_voc_path"])
        self.db = bow.Database()
        self.db.setVocabulary(voc, False, 0)
        self.load_local_descriptors()
        self.set_database()
        self.load_map()
        camera_model, width, height, params = self.config["camera_params"]
        self.camera_params = {
                        'model': camera_model,
                        'width': width,
                        'height': height,
                        'params': params,
                    }
        self.superpoint = sp(self.config["superpoint_config"])
        self.subscriber = rospy.Subscriber("/camera/image_raw",
                                          Image, self.callback, queue_size=500)
        
        
    def load_map(self):
        assert Path(self.config["sfm_dir"]).exists(), Path(self.config["sfm_dir"])
        logging.info('Reading 3D model...')
        _, self.db_images, self.points3D = read_model(str(self.config["sfm_dir"]), '.bin')
    

    def set_database(self):
        self.name_of_images_map = list(self.local_feature_file.keys())
        for i in self.name_of_images_map:
            d = np.transpose((self.local_feature_file[i]["descriptors"]).__array__())
            self.db.add(d)
        
    def load_local_descriptors(self):
        local_feature_path = Path(self.config["output_path"])/"local_feature.h5"
        self.local_feature_file = h5py.File(str(local_feature_path),"r")      
        
        
    def callback(self, image_data):
        self.start_time = time.time()
        self.cv_image = np.frombuffer(image_data.data, dtype=np.uint8).reshape(image_data.height, image_data.width, -1)
#         self.cv_image = cv2.cvtColor(self.cv_image, cv2.COLOR_BGR2RGB) changed in superpoint class
        self.pred = self.superpoint.key_point_and_descriptor(self.cv_image)
#         self.keypoints = self.pred["keypoints"]
#         self.local_descriptors = self.pred["descriptors"]
#         self.scores = self.pred["scores"]
        self.image_size = np.array(self.cv_image.shape[:2][::-1])
        self.find_global()
        
    def find_global(self):
        neighbours = self.db.query(np.transpose(self.pred["descriptors"]), self.config["global_matches"], -1)
        self.global_matches_name = [self.name_of_images_map[i.Id] for i in neighbours]
        self.find_superglue_matches()
    
    @torch.no_grad()
    def find_superglue_matches(self):
        self.match_dict = {}
        batches = {}
        for i in range(0,self.config["global_matches"], self.config["batch_size"]):
            batches[i] = self.global_matches_name[i:i+self.config["batch_size"]]
        for i in batches:
            names = batches[i]
            kplist0 = []
            kplist1 = []
            desc0 = []
            desc1 = []
            sc0 = []
            sc1 = []
            for j in names:
                pair = names_to_pair("query", j)
                feats1 = self.local_feature_file[j]
                kplist0.append(self.pred["keypoints"])
                kplist1.append(feats1["keypoints"].__array__())
                desc0.append(self.pred["descriptors"])
                desc1.append(feats1["descriptors"].__array__())
                sc0.append(self.pred["scores"])
                sc1.append(feats1["scores"].__array__())
            # pad feature0 not necessary
            size_list=[n.shape[0] for n in kplist1]
            max_size = np.max(size_list)
            kplist1 = [ np.concatenate((n, np.zeros((max_size-n.shape[0], n.shape[1]))), axis=0) for n in kplist1]
            desc1 = [ np.concatenate((n, np.zeros((n.shape[0], max_size-n.shape[1]))), axis=1) for n in desc1]
            sc1 = [ np.concatenate((n, np.zeros((max_size-n.shape[0]))), axis=0) for n in sc1]
            data = {'keypoints0':kplist0, 'descriptors0':desc0, 'scores0':sc0,'keypoints1':kplist1, 'descriptors1':desc1, 'scores1':sc1}
            data = {k: torch.from_numpy(np.array(v)).float().to(self.device) for k, v in data.items()}
            # some matchers might expect an image but only use its size
            data['image0'] = torch.empty((len(sc0), 1,)+tuple(self.image_size)[::-1])
            data['image1'] = torch.empty((len(sc0), 1,)+tuple(feats1['image_size'])[::-1])
            pred = self.superglue_model(data)
            index = 0 
            for k in names:
#                 pair = names_to_pair("query", k)
                Matches = pred["matches0"][index].cpu().short().numpy()
                if 'matching_scores0' in pred:
                    Scores = pred['matching_scores0'][index].cpu().half().numpy()
                self.match_dict[k] = {"scores":Scores, "matches":Matches}
                index+=1 
        self.find_pose()
        
    def find_pose(self):
        kp_idx_to_3D = defaultdict(list)
        kp_idx_to_3D_to_db = defaultdict(lambda:defaultdict(list))
        num_matches = 0
        for i,db_name in enumerate(self.global_matches_name):
            db_id = self.db_images[db_name].id
            points3D_ids = self.db_images[db_name].point3D_ids
            matches = self.match_dict[db_name]['matches']
            valid = np.where(matches>-1)[0] # the ones which were matched
            valid = valid[points3D_ids[matches[valid]]!=-1]
            num_matches +=len(valid)
            
            for idx in valid:
                id_3D = points3D_ids[matches[idx]]
                kp_idx_to_3D_to_db[idx][id_3D].append(i)
                # avoid duplicate observations
                if id_3D not in kp_idx_to_3D[idx]:
                    kp_idx_to_3D[idx].append(id_3D)
                    
        idxs = list(kp_idx_to_3D.keys())
        mkp_idxs = [i for i in idxs for _ in kp_idx_to_3D[i]]
        mkpq = self.pred["keypoints"][mkp_idxs]
        mkpq = mkpq.astype('float64')
        mkpq += 0.5  # COLMAP coordinates
        mp3d_ids = [j for i in idxs for j in kp_idx_to_3D[i]]
        mp3d = [self.points3D[j].xyz for j in mp3d_ids]
        mp3d = np.array(mp3d).reshape(-1, 3)
        ret = pycolmap.absolute_pose_estimation(mkpq, mp3d, self.camera_params, self.config["ransac_thresh"])
        if ret["success"]:
            poses = (ret["qvec"], ret["tvec"])
            tvec = cf.colmap_to_global(ret["tvec"], ret["qvec"])
#             print(tvec)
            print(time.time()- self.start_time)
#             print(poses)
        else:
            print("Failed")   
    
    
            
        

In [ ]:
config = {"superpoint_voc_path":"/home/Hierarchical-Localization/superpoint_voc.yml.gz",
          "superpoint_config":{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},'preprocessing': {'grayscale': True, 'resize_max': 960}},
          "output_path":"/home/Hierarchical-Localization/outputs/model_T3_localizedT4",
          "global_matches":5,
          "batch_size":5,
          'superglue_config':{'model': {'name': 'superglue', 'weights': 'outdoor', 'sinkhorn_iterations': 50}},
          "ransac_thresh":12,
          "sfm_dir":"/home/Hierarchical-Localization/outputs/model_T3_localizedT4/sfm_superpoint+superglue/geo_registered_model",
          "camera_params":("SIMPLE_RADIAL", 640, 480, (658.503, 320, 180, 0.0565491))}


In [ ]:
def main(config):
    ic = image_pose(config)
    rospy.init_node("image_pose", anonymous=True)
    rospy.spin()

In [ ]:
main(config)

In [ ]:
# Initializing superpoint
# Loaded SuperPoint model
# [ 0.0979085  -3.03516345  0.96791384]
# 0.46712589263916016
# [ 0.09815932 -3.03505715  0.96705313]
# 0.39977192878723145
# [ 0.09577007 -3.03372917  0.96710903]
# 0.39513659477233887
# [ 0.09833162 -3.0339158   0.96720681]
# 0.4117920398712158
# [ 0.09745279 -3.03418005  0.96745612]
# 0.3872103691101074
# [ 0.09734967 -3.03384185  0.96689259]
# 0.35898470878601074
# [ 0.09917752 -3.03221621  0.96805268]
# 0.34864282608032227
# [ 0.10070238 -3.02956419  0.96640818]
# 0.3467831611633301
# [ 0.10075921 -3.02431107  0.96959541]
# 0.3643312454223633
# [ 0.10140486 -3.0226479   0.9676392 ]
# 0.35076403617858887
# [ 0.10992665 -3.02378629  0.96633436]
# 0.3056032657623291
# [ 0.11600313 -3.0373605   0.96740127]
# 0.3783752918243408
# [ 0.12000719 -3.04475926  0.96759363]
# 0.3124096393585205
# [ 0.12333876 -3.0479677   0.96811521]
# 0.317349910736084
# [ 0.1304384  -3.05326279  0.96846136]
# 0.34629321098327637
# [ 0.12691022 -3.05465962  0.96640298]
# 0.3444089889526367
# [ 0.13136199 -3.06103463  0.96623683]
# 0.3154566287994385
# [ 0.13697064 -3.06772174  0.96718387]
# 0.3300290107727051
# [ 0.1377577  -3.07269885  0.96705837]
# 0.36144256591796875
# [ 0.13595192 -3.06624805  0.96707565]
# 0.35327816009521484
# [ 0.13454695 -3.0554303   0.96801047]
# 0.3065330982208252
# [ 0.13138994 -3.03724195  0.96693488]
# 0.367525577545166
# [ 0.12892383 -3.02445957  0.96766466]
# 0.34462547302246094
# [ 0.12748134 -3.0192815   0.96617943]
# 0.34714841842651367
# [ 0.12711912 -3.02056156  0.9668354 ]
# 0.36188220977783203
# [ 0.12737607 -3.02168475  0.96606674]
# 0.35420989990234375
# [ 0.12624942 -3.02117419  0.96739555]
# 0.3802309036254883
# [ 0.12904695 -3.02215829  0.96592599]
# 0.35496973991394043
# [ 0.12989998 -3.0246963   0.96830049]
# 0.34888339042663574
# [ 0.12757318 -3.02297743  0.96833089]
# 0.40122056007385254
# [ 0.12669276 -3.02380595  0.96814696]
# 0.3914792537689209
# [ 0.12721738 -3.02419475  0.96826883]
# 0.3469095230102539
# [ 0.12795624 -3.02352089  0.96631391]
# 0.3712921142578125
# [ 0.12967478 -3.02232143  0.96583189]
# 0.3808736801147461
# [ 0.13171018 -3.02541746  0.96621601]
# 0.3902463912963867
# [ 0.12848924 -3.02565616  0.96820315]
# 0.386732816696167
# [ 0.1294027  -3.02223438  0.96451591]
# 0.3975532054901123
# [ 0.13042005 -3.02568047  0.96558777]
# 0.3430163860321045
# [ 0.12884712 -3.02635918  0.96831242]
# 0.353515625
# [ 0.13081778 -3.0280244   0.96848994]
# 0.37058258056640625
# [ 0.12989283 -3.02655643  0.9676682 ]
# 0.4128298759460449
# [ 0.1276206  -3.02444663  0.96598822]
# 0.38869404792785645
# [ 0.12714754 -3.02488315  0.96571385]
# 0.35209131240844727
# [ 0.12790377 -3.02357359  0.9645519 ]
# 0.38242340087890625
# [ 0.1312876  -3.02392198  0.965312  ]
# 0.36111021041870117
# [ 0.12979718 -3.02276571  0.965708  ]
# 0.3678758144378662
# [ 0.12890367 -3.02531821  0.96770335]
# 0.3790771961212158
# [ 0.13193281 -3.02486871  0.96780593]
# 0.35359883308410645
# [ 0.13199061 -3.02590187  0.96621646]
# 0.3622605800628662
# [ 0.12601428 -3.02133795  0.96641149]
# 0.379608154296875
# [ 0.12963325 -3.02619357  0.96748708]
# 0.39327073097229004
# [ 0.12889863 -3.02455152  0.96622868]
# 0.352384090423584
# [ 0.12953163 -3.02528891  0.96485708]
# 0.359546422958374
# [ 0.12982556 -3.02597052  0.96773537]
# 0.34209394454956055
# [ 0.1336272  -3.02769771  0.96735662]
# 0.3509206771850586
# [ 0.12959978 -3.02539913  0.9662277 ]
# 0.37645554542541504
# [ 0.1272092  -3.0227502   0.96701338]
# 0.36627197265625
# [ 0.12960387 -3.02265076  0.96810556]
# 0.36319589614868164
# [ 0.12626668 -3.01793918  0.96792908]
# 0.3771793842315674
# [ 0.12823014 -3.01676287  0.96701331]
# 0.3478977680206299
# [ 0.12387343 -3.00784998  0.96602412]
# 0.35367560386657715
# [ 0.12510445 -3.00563448  0.96631719]
# 0.3606758117675781
# [ 0.12761932 -3.00797284  0.96599457]
# 0.33911776542663574
# [ 0.1234239  -3.00925286  0.96757327]
# 0.3727073669433594
# [ 0.12295795 -3.00403937  0.96917016]
# 0.3974745273590088
# [ 0.12336438 -2.99892395  0.96754974]
# 0.36010074615478516
# [ 0.12385563 -2.9982367   0.96453525]
# 0.3539738655090332
# [ 0.12338826 -2.99988908  0.96779261]
# 0.362626314163208
# [ 0.12208212 -2.99674207  0.96711188]
# 0.36894965171813965
# [ 0.12191193 -2.99130076  0.96628131]
# 0.3451251983642578
# [ 0.11985119 -2.98549728  0.96701669]
# 0.3248116970062256
# [ 0.11921282 -2.97850214  0.96881685]
# 0.3231053352355957
# [ 0.12034023 -2.97061072  0.96688803]
# 0.30494165420532227
# [ 0.12020785 -2.96934378  0.97082143]
# 0.34421634674072266
# [ 0.11821005 -2.97852721  0.9686249 ]
# 0.3460838794708252
# [ 0.12533051 -2.9771349   0.96725982]
# 0.29200124740600586
# [ 0.12700846 -2.99085173  0.96985103]
# 0.2892882823944092
# [ 0.13524605 -3.01077186  0.96826586]
# 0.29514384269714355
# [ 0.13136365 -3.0186994   0.9674214 ]
# 0.3134739398956299
# [ 0.13758953 -3.03457543  0.96752074]
# 0.355236291885376
# [ 0.1420971  -3.05505199  0.96720754]
# 0.2954850196838379
# [ 0.1436961  -3.08838715  0.96950446]
# 0.32104015350341797
# [ 0.14444464 -3.13620291  0.97012237]
# 0.3113596439361572
# [ 0.14267296 -3.19643659  0.96961138]
# 0.34200453758239746
# [ 0.14107558 -3.25314818  0.97091775]
# 0.3442718982696533
# [ 0.14025564 -3.30155399  0.97096184]
# 0.34266090393066406
# [ 0.13738303 -3.3413852   0.97187105]
# 0.32921576499938965
# [ 0.13696143 -3.37580638  0.97063796]
# 0.3390190601348877
# [ 0.13713453 -3.40178274  0.97155163]
# 0.34943509101867676
# [ 0.13530047 -3.41784932  0.9737621 ]
# 0.3707420825958252
# [ 0.13432358 -3.42091674  0.97288593]
# 0.3705136775970459
# [ 0.13491146 -3.4178466   0.9739626 ]
# 0.34214043617248535
# [ 0.13612556 -3.42601933  0.9722839 ]
# 0.34217000007629395
# [ 0.1316649  -3.4458581   0.97373647]
# 0.3105959892272949
# [ 0.12979572 -3.47864621  0.97405832]
# 0.2929248809814453
# [ 0.12874172 -3.49423213  0.97437322]
# 0.3504002094268799
# [ 0.13141132 -3.4903856   0.97526554]
# 0.3606743812561035
# [ 0.12886532 -3.46377511  0.97262481]
# 0.34123802185058594
# [ 0.12986039 -3.42946319  0.97181856]
# 0.35812926292419434
# [ 0.13109579 -3.39893062  0.97253383]
# 0.32294154167175293
# [ 0.131009   -3.37412777  0.97169869]
# 0.36272239685058594
# [ 0.13212493 -3.34850284  0.97065976]
# 0.3589975833892822
# [ 0.13115619 -3.32868248  0.96805958]
# 0.3700423240661621
# [ 0.13067534 -3.31374565  0.97100656]
# 0.3541114330291748
# [ 0.13042993 -3.3022247   0.96943129]
# 0.35492897033691406
# [ 0.12927986 -3.30417457  0.96984821]
# 0.3568277359008789
# [ 0.1295592  -3.29555993  0.97149919]
# 0.34243106842041016
# [ 0.12723046 -3.27106186  0.97018955]
# 0.4820973873138428
# [ 0.12599837 -3.25377045  0.97021543]
# 0.33171677589416504
# [ 0.12680681 -3.24095413  0.97003887]
# 0.3706197738647461
# [ 0.12748029 -3.215746    0.9688185 ]
# 0.34324121475219727
# [ 0.12439972 -3.1865404   0.96978631]
# 0.3692009449005127
# [ 0.12363155 -3.15976711  0.96875542]
# 0.3554191589355469
# [ 0.1258739  -3.12874193  0.96839942]
# 0.3404359817504883
# [ 0.12377094 -3.08627057  0.96815458]
# 0.3535919189453125
# main(config)


In [ ]:
# SP = sp()



# for i in image_dict:
#     image_array = image_dict[i]["image"]
#     pred = SP.key_point_and_descriptor(image_array)
#     print(np.shape(pred["descriptors"]))

    
    
        
# def processed(self):
#     if config["superpoint_config"]["preprocessing"]["grayscale"]:
#         self.cv_image = ((cv2.cvtColor(self.cv_image, cv2.COLOR_RGB2GRAY)).astype(np.float32))
#         ## omited the reshape function need to add it if needed
#         w, h = self.image_size
#     if config["superpoint_config"]["preprocessing"]["resize_max"] and max(w,h)> config["superpoint_config"]["preprocessing"]["resize_max"]:
#         scale = config["superpoint_config"]["preprocessing"]["resize_max"]/max(h,w)
#         h_new, w_new = int(round(h*scale)), int(round(w*scale))
#         self.cv_image = cv2.resize(self.cv_image, (w_new, h_new), interpolation=cv2.INTER_LINEAR)
#     if config["superpoint_config"]["preprocessing"]["grayscale"]:
#         self.cv_image = self.cv_image[None]
#     else:
#         self.cv_image = self.cv_image.transpose((2,0,1))
#         self.cv_image = self.cv_image/255.
# #         print(np.shape(self.cv_image))

#     self.image_data = {
#         'image': (torch.from_numpy(self.cv_image)).unsqueeze(0),
#         'original_size': torch.from_numpy(self.image_size)
#     }
#     self.keypoint_and_descriptors()



#     def load_superpoint(self):
#         Model_sp = dynamic_load(extractors, self.config["superpoint_config"]['model']['name'])
#         self.model_sp = Model_sp(self.config["superpoint_config"]['model']).eval().to(self.device)
 
    
    
    
#     @torch.no_grad()
#     def keypoint_and_descriptors(self):
#         pred_sp = self.model_sp(map_tensor(self.image_data, lambda x:x.to(self.device)))
#         print(self.image_data["image"].size())
#         pred_sp = {k: v[0].cpu().numpy() for k, v in pred_sp.items()}
#         pred_sp["image_size"] = original_size = self.image_data["original_size"][0].numpy()
# #         print(np.shape(pred_sp["keypoints"]))
        
#         if "keypoints" in pred_sp:
#             size = np.array(self.image_data['image'].shape[-2:][::-1])
#             scales = (original_size / size).astype(np.float32)
#             pred_sp['keypoints'] = (pred_sp['keypoints'] + .5) * scales[None] - .5
#             self.pred_sp = pred_sp
#             del pred_sp
#         self.find_global()